In [1]:
import os
from pathlib import Path
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
# Import necessary libraries

import re
import string
import dagshub
import mlflow
import mlflow.sklearn
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [5]:
base_dir = Path(os.getenv('BASE_DIR'))
base_data_dir = Path(os.getenv('BASE_DATA_DIR'))
external_data_path = os.path.join(base_data_dir,"external")
raw_data_path = os.path.join(base_data_dir, "raw")
notebooks_path = os.path.join(base_dir, "notebooks")
os.makedirs(raw_data_path, exist_ok=True)
os.makedirs(external_data_path, exist_ok=True)

In [6]:
mlflow.set_tracking_uri('https://dagshub.com/insculptor/mlops-mini-project.mlflow')
dagshub.init(repo_owner='insculptor', repo_name='mlops-mini-project', mlflow=True)

Accessing as insculptor

Initialized MLflow to track repo "insculptor/mlops-mini-project"

Repository insculptor/mlops-mini-project initialized!

In [7]:
df = pd.read_csv(os.path.join(external_data_path, 'tweet_emotions.csv')).drop(columns=['tweet_id'])
df.head(5)

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [8]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['content'] = df['content'].apply(lower_case)
        df['content'] = df['content'].apply(remove_stop_words)
        df['content'] = df['content'].apply(removing_numbers)
        df['content'] = df['content'].apply(removing_punctuations)
        df['content'] = df['content'].apply(removing_urls)
        df['content'] = df['content'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [9]:
df = normalize_text(df)
df.head(5)

,sentiment,content
0,empty,tiffanylue know listenin bad habit earlier sta...
1,sadness,layin n bed headache ughhhh waitin call
2,sadness,funeral ceremony gloomy friday
3,enthusiasm,want hang friend soon
4,neutral,dannycastillo want trade someone houston ticke...


In [10]:
x = df['sentiment'].isin(['sadness', 'happiness'])
df = df[x]
df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})
df.head()

C:\Users\erdrr\AppData\Local\Temp\ipykernel_14040\2264095480.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})


,sentiment,content
1,0,layin n bed headache ughhhh waitin call
2,0,funeral ceremony gloomy friday
6,0,sleep im not thinking old friend want he s mar...
8,0,charviray charlene love miss
9,0,kelcouch i m sorry least friday


In [11]:
# Set the experiment name
mlflow.set_experiment("MLops-Mini-Project_Exp2_BOW_TFIDF")

# Define feature extraction methods
vectorizers = {
    'BoW': CountVectorizer(),
    'TF-IDF': TfidfVectorizer()
}

2024/08/11 11:04:40 INFO mlflow.tracking.fluent: Experiment with name 'MLops-Mini-Project_Exp2_BOW_TFIDF' does not exist. Creating a new experiment.


In [12]:
# Define algorithms
algorithms = {
    'LogisticRegression': LogisticRegression(),
    'MultinomialNB': MultinomialNB(),
    'XGBoost': XGBClassifier(),
    'RandomForest': RandomForestClassifier(),
    'GradientBoosting': GradientBoostingClassifier()
}

In [13]:
with mlflow.start_run(run_name="All_Experiments") as run:
    ## Loop throiuugh the algorithms
    for algo_name, algo in algorithms.items():
        ## Loop through the feature extraction methods
        for vec_name, vectorizer in vectorizers.items():
            with mlflow.start_run(run_name=f"{algo_name} with {vec_name}",nested=True)  as child_run:
                X = vectorizer.fit_transform(df['content'])
                y = df['sentiment']
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
                
                ## Log Preprocessing Parameters
                mlflow.log_param("Vectorizer", vec_name)
                mlflow.log_param("Algorithm", algo_name)
                mlflow.log_param("Train-Test Split", 0.2)
                
                ## Model Training
                model = algo
                model.fit(X_train, y_train)
                
                ## Log model parameters
                mlflow.log_params(model.get_params())
                
                # MOdel Evaluation
                y_pred = model.predict(X_test)
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred)
                recall = recall_score(y_test, y_pred)
                f1 = f1_score(y_test, y_pred)
                
                ## Log Model Metrics
                mlflow.log_metric("Accuracy", accuracy)
                mlflow.log_metric("Precision", precision)
                mlflow.log_metric("Recall", recall)
                mlflow.log_metric("F1", f1)
                
                # Log Model
                mlflow.sklearn.log_model(model, f"{algo_name}_{vec_name}")
                
                ## Log Jupyter Notebook
                notebook_path = os.path.join(notebooks_path, "exp2_bow_vs_tfidf.ipynb")
                os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
                mlflow.log_artifact(notebook_path)
                print(f"Experiment Completed: {algo_name} with {vec_name}")
                print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1: {f1}")
                              
                
            

2024/08/11 11:13:21 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/11 11:13:24 INFO mlflow.tracking._tracking_service.client: 🏃 View run LogisticRegression with BoW at: https://dagshub.com/insculptor/mlops-mini-project.mlflow/#/experiments/2/runs/37666bfb2b3f41989a5cf4579299acba.
2024/08/11 11:13:24 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/insculptor/mlops-mini-project.mlflow/#/experiments/2.


Experiment Completed: LogisticRegression with BoW
Accuracy: 0.7946987951807228, Precision: 0.7862001943634597, Recall: 0.7970443349753694, F1: 0.7915851272015656


c:\Users\erdrr\anaconda3\envs\mlops_py311\Lib\site-packages\_distutils_hack\__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\erdrr\anaconda3\envs\mlops_py311\Lib\site-packages\_distutils_hack\__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
2024/08/11 11:13:30 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/11 11:13:33 INFO mlflow.tracking._tracking_service.client: 🏃 View run LogisticRegression with TF-IDF at: https://dagsh

Experiment Completed: LogisticRegression with TF-IDF
Accuracy: 0.7956626506024096, Precision: 0.7795648060548723, Recall: 0.8118226600985222, F1: 0.7953667953667953


c:\Users\erdrr\anaconda3\envs\mlops_py311\Lib\site-packages\_distutils_hack\__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\erdrr\anaconda3\envs\mlops_py311\Lib\site-packages\_distutils_hack\__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
2024/08/11 11:13:40 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/11 11:13:43 INFO mlflow.tracking._tracking_service.client: 🏃 View run MultinomialNB with BoW at: https://dagshub.com/i

...

2024/08/11 11:15:29 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/insculptor/mlops-mini-project.mlflow/#/experiments/2.
c:\Users\erdrr\anaconda3\envs\mlops_py311\Lib\site-packages\_distutils_hack\__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\erdrr\anaconda3\envs\mlops_py311\Lib\site-packages\_distutils_hack\__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
2024/08/11 11:15:49 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is 

Experiment Completed: GradientBoosting with BoW
Accuracy: 0.7291566265060241, Precision: 0.8073270013568521, Recall: 0.5862068965517241, F1: 0.6792237442922374


c:\Users\erdrr\anaconda3\envs\mlops_py311\Lib\site-packages\_distutils_hack\__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\erdrr\anaconda3\envs\mlops_py311\Lib\site-packages\_distutils_hack\__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
2024/08/11 11:16:14 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/11 11:16:17 INFO mlflow.tracking._tracking_service.client: 🏃 View run GradientBoosting with TF-IDF at: https://dagshub

Experiment Completed: GradientBoosting with TF-IDF
Accuracy: 0.7219277108433735, Precision: 0.8008241758241759, Recall: 0.574384236453202, F1: 0.6689615605278256


2024/08/11 11:16:17 INFO mlflow.tracking._tracking_service.client: 🏃 View run All_Experiments at: https://dagshub.com/insculptor/mlops-mini-project.mlflow/#/experiments/2/runs/98beb987477a42bbb249af14b65307a0.
2024/08/11 11:16:17 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/insculptor/mlops-mini-project.mlflow/#/experiments/2.
